In [ ]:
#!pip3 install --upgrade --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
#pip install -r requirements.txt

In [2]:
from data_util import get_data_from_kaggle
from pretrained_model import sentiment_analysis

c:\Users\Luan\Documents\DL\TFT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
stock, tweet = get_data_from_kaggle()

In [18]:
tsla = tweet[ tweet['Stock Name']=='TSLA'  ].loc[0]

In [20]:
tsla['sentiment'] = sentiment

In [4]:
sentiment = sentiment_analysis(tweets_arr=tweet[ tweet['Stock Name']=='TSLA'  ]['Tweet'])

c:\Users\Luan\Documents\DL\TFT\pretrained_model.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  scores = softmax(output[0][0])


In [21]:
tsla

Date                                    2022-09-29 23:41:16+00:00
Tweet           Mainstream media has done an amazing job at br...
Stock Name                                                   TSLA
Company Name                                          Tesla, Inc.
sentiment                   [[0.528979, 0.35103178, 0.119989194]]
Name: 0, dtype: object

In [6]:
tweet['sentiment'] = sentiment

ValueError: Length of values (1) does not match length of index (80793)

In [18]:
nvcc --version

NameError: name 'nvcc' is not defined

In [2]:
torch.cuda.is_available()

True

In [1]:
import torch